# Eploring data splits


## Load different data splits

In [15]:
# Set path
path = 'C:/Users/User/Desktop/Assignatures/Synthesis project/GraphAnomaly/dades_arnau/'

In [16]:
import pickle

# Load datasets. Both train and test sets are on the same file, stored as a tuple
with open(path + 'originals.pkl', 'rb') as f:
    originals = pickle.load(f)
with open(path + 'originalb.pkl', 'rb') as f:
    originalb = pickle.load(f)
with open(path + 'nodes.pkl', 'rb') as f:
    originali = pickle.load(f)

In [17]:
from sklearn.model_selection import train_test_split

originali = originali.loc[originali['class'].isin([0, 1])] # Drop unknown
y_traini = list(originali['class'])
X_traini = originali.drop(columns=['class'])

isolated = train_test_split(X_traini, y_traini, test_size=0.2, random_state=42)

## Testing

In [18]:
from sklearn.metrics import *

def metrics(name, y_test, y_pred, df_results):    
    # Except for the accuracy, the others compute the metric for the ilicit class    
    accuracy = round(accuracy_score(y_test, y_pred) * 100, 4)    
    precision = round(precision_score(y_test, y_pred, pos_label=0) * 100, 4)    
    recall = round(recall_score(y_test, y_pred, pos_label=0) * 100, 4)    
    f1 = round(f1_score(y_test, y_pred, pos_label=0) * 100,4)

    df_results.loc[len(df_results)] = [name, accuracy, precision, recall, f1] 
    return df_results   
    

In [19]:
from Helper import separate_labels

def test_split(name, split, model, df_results):
    # Split labels
    if not name=='Isolated':
        X_train, y_train, X_test, y_test = separate_labels(split[0], split[1])
    else:
        X_train, X_test, y_train, y_test = isolated
    # Feed data into model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)    
    df_results = metrics(name, y_test, y_pred, df_results)
    return df_results

In [21]:
import pandas as pd
from sklearn.ensemble import *

df_results = pd.DataFrame(columns=["Split", "Accuracy", "Precision", "Recall", "F1 score"])
splits = [originals, originalb, isolated]
names = ['Sequential', 'Balanced', 'Isolated']

# We use random forest since it is the best performing ML method
for name, split in zip(names, splits):
    df_results = test_split(name, split, RandomForestClassifier(), df_results)

df_results

,Split,Accuracy,Precision,Recall,F1 score
0,Sequential,97.9712,98.0456,66.8147,79.4719
1,Balanced,98.2366,99.3590,86.7684,92.6379
2,Isolated,98.8725,99.6250,88.6541,93.8199
